In [1]:
import pydicom as dcm
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog

#Get the path of the folder containing the DICOM files
root = tk.Tk()
root.withdraw()
path = filedialog.askdirectory()
mascara = filedialog.askopenfilename()

#Check if the path is valid
if os.path.isdir(path):
    print("Path is valid")
else:
    print("Path is not valid")
    raise ValueError("Path is not valid")

#Check if the path is valid
if os.path.isfile(mascara):
    print("Path is valid")
else:
    print("Path is not valid")
    raise ValueError("Path is not valid")


In [4]:
path =r"D:\Documentos\CUCEI\6t0\Procesamiento\images\DICOM\DICOM-20210818T133238Z-001\DICOM"
mascara:['D:\\Documentos\\CUCEI\\6t0\\Procesamiento\\images\\DICOM…']=r"D:\Documentos\CUCEI\6t0\Procesamiento\images\DICOM\DICOM-20210818T133238Z-001\mascaras\mask.nrrd"

In [5]:
# Read all files in folder
files = os.listdir(path)
# Join the path with the name of the file
listed_files = [os.path.join(path, file) for file in files]

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\Documentos\\CUCEI\\6t0\\Procesamiento\\images\\DICOM\\DICOM-20210818T133238Z-001\\DICOM'

In [6]:
# Sort the files by InstanceNumber
sorted = listed_files.sort(key=lambda x: int(dcm.read_file(x).get('InstanceNumber', 0))) 


NameError: name 'listed_files' is not defined

In [ ]:
#Get image size
rows = int(dcm.read_file(listed_files[0]).Rows)
cols = int(dcm.read_file(listed_files[0]).Columns)
slices = len(listed_files)

In [ ]:
#Create a numpy array with the size of the image
image = np.zeros([rows, cols, slices])

In [ ]:
#Read the files and store them in the numpy array ordered by InstanceNumber
image = np.stack([dcm.read_file(file).pixel_array for file in listed_files], axis=2)

In [ ]:
#Read nrrd mask
import nrrd
mask = nrrd.read(mascara)

In [ ]:
#Get the mask 
mask = mask[0]

In [ ]:
#Flip the mask to match the image
image = np.flip(image, axis=2)

In [ ]:
#Convert image to Hounsfield Units
HUimage = image * dcm.read_file(listed_files[0]).RescaleSlope + dcm.read_file(listed_files[0]).RescaleIntercept

In [ ]:
#Eliminate the values abovve 400 HU
HUimage[HUimage < -100] = 0

In [ ]:
#Normalize image from 0 to 100 
Norm_image = (HUimage - np.min(HUimage)) / (np.max(HUimage) - np.min(HUimage)) * 100

In [ ]:
#For each pixel in the mask with value 1, set the value of the image to 255
for i in range(0, mask.shape[0]):
    for j in range(0, mask.shape[1]):
        for k in range(0, mask.shape[2]):
            if mask[i,j,k] == 1:
                Norm_image[i,j,k] = 255

In [ ]:
#Apply mask as a filter
masked = Norm_image

In [29]:
#Create a Max intensity projection
mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
mlab.contour3d(HUimage, contours=10, transparent=True, opacity=0.5)
mlab.outline()
mlab.axes()
mlab.show()

In [ ]:
mlab.figure(bgcolor=(0, 0, 0), size=(800, 800))
vol = mlab.pipeline.scalar_field(HUimage)
mlab.pipeline.volume(vol, vmin=0)
mlab.show()